## 실습02번에서 랜덤하게 뽑은 맛집을 지도에 시각화해봅시다

#### 지도에 시각화하려면 어떤 정보가 필요할까요?

저희가 평소 이용하는 네이버 지도, 구글맵 등은 지역포인트를 위/경도로 표시합니다.

저희도 이처럼 데이터를 위/경도로 변환하는 여러 방법을 알아보도록 합시다.

In [ ]:
import sys
import time
from selenium import webdriver
from selenium.common import exceptions

# 데이터 수집을 위한 망고플레이트 크롤링

#### 브라우저 객체 생성 

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.mangoplate.com/")

#### 망고플레이트 검색창 엘리먼트 탐색 및 검색어 입력

In [ ]:
search_element = driver.find_element_by_css_selector('#main-search')
search_element.clear()
search_element.send_keys('한대앞역')

search_button = driver.find_element_by_css_selector(".btn-search")
search_button.click()

# 식당 상호명 리스트 생성

In [ ]:
restaurant_list = driver.find_element_by_css_selector('ul.list-restaurants')
restaurant_info = restaurant_list.find_elements_by_css_selector('figure.restaurant-item')

total_restaurant_info_list = []
for i in restaurant_info[:10]:
    try:
        name, address = i.find_element_by_css_selector('.thumb img').get_attribute('alt').split('사진 -')
        total_restaurant_info_list.append((name.strip(), address.strip()))
    except(exceptions.NoSuchElementException):
        break

# 오류수정 17.10.10
# 한대앞역 으로 검색했을때는 여러 페이지 등장 했으나, 다른 검색어 '안산 한양대' 의 경우에는
#  1페이지 밖에 없어서 .paging 엘리먼트들이 없어서 ElementNotVisibleException 발생.
# 만약, 이 오류라면 pass 하는 것으로 로직변경
try:
    page_nums = driver.find_element_by_css_selector('.paging').find_elements_by_css_selector('.ng-binding.ng-scope')[1:]

    cnt = 0
    for p in page_nums:
        cnt += 1
        sys.stdout.write('\r%d / %d' % (cnt, len(page_nums)))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        p.click()
        time.sleep(5)
        restaurant_list = driver.find_element_by_css_selector('ul.list-restaurants')
        restaurant_info = restaurant_list.find_elements_by_css_selector('figure.restaurant-item')
        for i in restaurant_info[:10]:
            name, address = i.find_element_by_css_selector('.thumb img').get_attribute('alt').split('사진 -')
            total_restaurant_info_list.append((name.strip(), address.strip()))
except(exceptions.ElementNotVisibleException) as e:
    pass

print('END!')

#### 한대앞역 망고플레이트 맛집 상호명과 주소

In [ ]:
total_restaurant_info_list[:3]

#### Pandas 로 csv파일로 만듭시다

In [ ]:
import pandas as pd

In [ ]:
restaurant_df = pd.DataFrame(total_restaurant_info_list, columns=['name', 'address'])

In [ ]:
restaurant_df.to_csv('./한대앞_망고플레이트_맛집목록.csv', index=False, encoding='utf8')

In [ ]:
restaurant_df

#### 1. Geocode by Awesome Table 사용

특강 시, 진행예정

In [ ]:
# 특강에서 설명할 적용방법을 진행 후, 아래의 코드 주석을 해제해주세요.

# restaurant_df = pd.read_csv('./restraunt_list.csv')
# restaurant_df.head(5)
# restaurant_df.columns = ['name', 'address', 'latitude', 'longitude']

#### 2. google geocode API를 사용

In [ ]:
import requests
import json

def get_latitude_longitude(addr):
    '''
    DESCRIPTION: 주소를 입력하면 위도(latitude)/경도(longitude)를 반환하는 함수 
    ex) get_latitude_longitude('경기도 안산시 상록구 한양대학로 55')
    '''
    location = addr
    response = requests.get('http://maps.googleapis.com/maps/api/geocode/json?sensor=false&language=ko&address=' + location)    
    loc_dict = json.loads(response.text)
    latitude = loc_dict['results'][0]['geometry']['location']['lat'] 
    longitude = loc_dict['results'][0]['geometry']['location']['lng']
    return [latitude, longitude]

In [ ]:
latitude_list = []
longitude_list = []
for i in restaurant_df['address'].values:
    lat, lon = get_latitude_longitude(i)
    latitude_list.append(lat)
    longitude_list.append(lon)

restaurant_df['latitude'] = latitude_list
restaurant_df['longitude'] = longitude_list

# 수정 17.10.10 - API이용 버전 위/경도 반영 DataFrame 저장
restaurant_df.to_csv('./한대앞_망고플레이트_맛집목록_위경도.csv', index=False, encoding='utf8')

In [ ]:
restaurant_df.head(20)

In [ ]:
import folium

# 오류수정 17.10.10 - Latitude, Longitude를 latitude, longitude로 변경. 
hanyang_univ_map = folium.Map(location=[  37.298524, 126.834712], zoom_start=15)
for l in restaurant_df[['name', 'latitude', 'longitude']].values:
#    folium.Marker(l[1:],popup=l[0], icon=folium.Icon(color='red')).add_to(한양대지도)
    folium.CircleMarker(l[1:], popup=l[0], color='red', fill_color='red', radius=5).add_to(hanyang_univ_map)

In [ ]:
hanyang_univ_map

# 식당 랜덤하게 뽑기

In [ ]:
import random
# 식당 랜덤뽑기
rand_restaurant = random.choice(total_restaurant_info_list)[0]
# 식당정보 데이터프레임에서 뽑은 식당 정보 추출
print(restaurant_df[restaurant_df['name'] == rand_restaurant])
rand_latlng = restaurant_df[restaurant_df['name'] == rand_restaurant][['latitude', 'longitude']].values[0]
print(rand_latlng)

In [ ]:
# 지도 
# 오류 수정 17.10.10 - 지도 마커가 색칠되지않아 color를 red로 변경

hanyang_univ_map = folium.Map(location=[  37.300853, 126.842930], zoom_start=15)
folium.CircleMarker(rand_latlng, color='red', fill_color='red', radius=5).add_to(hanyang_univ_map)
hanyang_univ_map